In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content
# Install mmcv
!pip install mmcv -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
# Clone mmcls repository
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification/
# Install MMClassification from source
!pip install -e . 

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 554 kB 32.3 MB/s 
     |████████████████████████████████| 190 kB 62.3 MB/s 
  Created wheel for mmcv: filename=mmcv-1.6.0-py2.py3-none-any.whl size=848005 sha256=31770f5795d3a6b3d96ce08a179cbeb189b336d5d6eaef88a093e42ebd1d9c16
  Stored in directory: /root/.cache/pip/wheels/bc/d3/f7/5ad65ec39feb2546b12183f826ffe6bfbbd8a56aaa389111be
Successfully built mmcv
Cloning into 'mmclassification'...
remote: Enumerating objects: 8712, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 8712 (delta 0), reused 3 (delta 0), pack-reused 8701
Receiving objects: 100% (8712/8712), 9.87 MiB | 10.20 MiB/s, done.
Resolving deltas: 100% (6073/6073), done.
/content/mmclassification
Looking in indexes: https://pypi.o

In [3]:
# Check MMClassification installation
import torch, torchvision
import mmcls
import mmcv
from mmcls.apis import inference_model, init_model, show_result_pyplot
from mmcv import Config
from mmcls.apis import set_random_seed
import time
import os.path as osp
from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model
import gc
print(mmcls.__version__)

/usr/local/lib/python3.7/dist-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


0.23.1


In [4]:
def create_model(config_file,checkpoint_file,datasetpath,epochs=3,train_cfg=None):
  # Load the base config file
  cfg = Config.fromfile(config_file)

  # Modify the number of classes in the head.
  cfg.model.head.num_classes = 3

  # if custom train cfg defined ( for swin transformers we must define it)
  if train_cfg!=None:
    cfg.model.train_cfg=train_cfg

  cfg.model.head.topk = (1, )

  # Load the pre-trained model's checkpoint.
  cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')

  # Specify sample size and number of workers.
  cfg.data.samples_per_gpu = 8
  cfg.data.workers_per_gpu = 2

  # Specify the path and meta files of training dataset
  cfg.data.train.data_prefix = datasetpath+'/train_set/train_set'
  cfg.data.train.classes = datasetpath+'/clasess.txt'

  # Specify the path and meta files of validation dataset
  cfg.data.val.data_prefix = datasetpath+'/val_set/val_set'
  cfg.data.val.ann_file = datasetpath+'/val.txt'
  cfg.data.val.classes = datasetpath+'/clasess.txt'

  # Specify the path and meta files of test dataset
  cfg.data.test.data_prefix = datasetpath+'/test_set/test_set'
  cfg.data.test.ann_file = datasetpath+'/test.txt'
  cfg.data.test.classes = datasetpath+'/clasess.txt'


  # Modify the evaluation metric
  cfg.evaluation['metric_options']={'topk': (1, )}

  # Specify the optimizer
  cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
  cfg.optimizer_config = dict(grad_clip=None)

  # Specify the learning rate scheduler
  cfg.lr_config = dict(policy='step', step=1, gamma=0.1)
  cfg.runner = dict(type='EpochBasedRunner', max_epochs=epochs)

  # Specify the work directory
  cfg.work_dir = './work_dirs/Covid19-dataset'

  # Output logs for every 10 iterations
  cfg.log_config.interval = 10

  # Set the random seed and enable the deterministic option of cuDNN
  # to keep the results' reproducible.
  
  cfg.seed = 0
  set_random_seed(0, deterministic=True)

  cfg.gpu_ids = range(1)
  return cfg

In [5]:
def train(cfg):
  # Create the work directory
  mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
  # Build the classifier
  model = build_classifier(cfg.model)
  model.init_weights()
  # Build the dataset
  datasets = [build_dataset(cfg.data.train)]
  # Add `CLASSES` attributes to help visualization
  model.CLASSES = datasets[0].CLASSES
  # Start fine-tuning
  train_model(
      model,
      datasets,
      cfg,
      distributed=False,
      validate=True,
      timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
      meta=dict())
  gc.collect()


# Efficient-net

In [ ]:
cfg_model_efficientnet=create_model(config_file='configs/efficientnet/efficientnet-b0_8xb32_in1k.py',
                       checkpoint_file='https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32_in1k_20220119-a7e2a0b1.pth',
                       datasetpath='/content/drive/MyDrive/test_data/Covid19-dataset/Covid19-dataset',
                       epochs=30
)
train(cfg=cfg_model_efficientnet)

# Vision transformer

In [ ]:
cfg_model_vision=create_model(config_file='configs/vision_transformer/vit-base-p16_ft-64xb64_in1k-384.py',
                       checkpoint_file='https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth',
                       datasetpath='/content/drive/MyDrive/test_data/Covid19-dataset/Covid19-dataset',
                       epochs=30
)
train(cfg=cfg_model_vision)

# Swin transformer

In [ ]:
cfg_model_swin=create_model(config_file='configs/swin_transformer/swin-small_16xb64_in1k.py',
                       checkpoint_file='https://download.openmmlab.com/mmclassification/v0/swin-transformer/convert/swin_small_patch4_window7_224-cc7a01c9.pth',
                       datasetpath='/content/drive/MyDrive/test_data/Covid19-dataset/Covid19-dataset',
                       epochs=30,
                       train_cfg=dict(augments=[
                                      dict(type='BatchMixup', alpha=0.8, num_classes=3, prob=0.5),
                                      dict(type='BatchCutMix', alpha=1.0, num_classes=3, prob=0.5)
                                      ])
)
train(cfg=cfg_model_swin)